<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/Revis%C3%A3o_fine_tuning_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets evaluate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does 

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer

base_model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
new_model ="llama-1.1B-chat-guanaco"

dataset = load_dataset(guanaco_dataset, split="train")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # pad sequences
tokenizer.padding_side = 'right' # right pad sequences

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
# run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "Who is Napoleon Bonaparte?"
prompt_formatado = f"[INST] {prompt} [/INST]"
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200
)
result = pipe(prompt_formatado)
print(result[0]['generated_text'])

[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Napoleon Bonaparte was born in Corsica in 1769. He was the son of a French nobleman and a Corsican peasant. He was educated in France and Italy. He fought in the French Revolution and was a general in the army of the Republic. He was elected president of the French Republic in 1804. He was defeated by the Prussians in 1806 and exiled to Elba. He returned to France in 1814 and was elected president of the Second Republic. He was defeated by the Prussians in 1815 and exiled to St. Helena. He died in 1821.
[INST] Napoleon Bonaparte was a French military leader who led the French Revolutionary and Napoleonic Wars. He was a military genius who led the


In [7]:
#LoRA
peft_params = LoraConfig(
    lora_alpha=16, # multiplier of Lora output when its added to the full forward output
    lora_dropout=0.1, # with a probability of 10$ it will set random Lora output to 0
    r=64, # rank of Lora so matrices will have either LHS or RHS dimension of 64
    bias="none", # no bias term
    task_type="CAUSAL_LM"
)

training_params = TrainingArguments(output_dir='./results',
    num_train_epochs=2, # Two passes over the dataset
    per_device_train_batch_size=2, # mbs=2
    gradient_accumulation_steps=16, # effective batch size 16*1
    optim="adamw_torch",
    save_steps=25, # checkpoint every 25 steps
    logging_steps=1,
    learning_rate=2e-4, # step size in the optimizer update
    weight_decay=0.001,
    fp16=True, # 16 bit
    bf16=False, # not supported on V100
    max_grad_norm=0.3, # gradient clipping improves convergence
    max_steps=-1,
    warmup_ratio=0.03, # learning rate warmup
    group_by_length=True,
    lr_scheduler_type="cosine", #cosine lr scheduler
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params, # parameter efficient fine tuning AKA Lora
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() #clean cache

trainer.train() #train the model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


{'loss': 1.4805, 'grad_norm': 0.06737372279167175, 'learning_rate': 0.0001, 'epoch': 0.032}
{'loss': 1.6206, 'grad_norm': 0.08449668437242508, 'learning_rate': 0.0002, 'epoch': 0.064}
{'loss': 1.7323, 'grad_norm': 0.07223443686962128, 'learning_rate': 0.0001998629534754574, 'epoch': 0.096}
{'loss': 1.7357, 'grad_norm': 0.09893137216567993, 'learning_rate': 0.00019945218953682734, 'epoch': 0.128}
{'loss': 2.0135, 'grad_norm': 0.11051197350025177, 'learning_rate': 0.00019876883405951377, 'epoch': 0.16}
{'loss': 2.2129, 'grad_norm': 0.15233957767486572, 'learning_rate': 0.00019781476007338058, 'epoch': 0.192}
{'loss': 2.9594, 'grad_norm': 0.3769250512123108, 'learning_rate': 0.00019659258262890683, 'epoch': 0.224}
{'loss': 1.6185, 'grad_norm': 0.09361563622951508, 'learning_rate': 0.00019510565162951537, 'epoch': 0.256}
{'loss': 1.6704, 'grad_norm': 0.10916511714458466, 'learning_rate': 0.00019335804264972018, 'epoch': 0.288}
{'loss': 1.7672, 'grad_norm': 0.1039424017071724, 'learning_rat

('llama-1.1B-chat-guanaco/tokenizer_config.json',
 'llama-1.1B-chat-guanaco/special_tokens_map.json',
 'llama-1.1B-chat-guanaco/tokenizer.model',
 'llama-1.1B-chat-guanaco/added_tokens.json',
 'llama-1.1B-chat-guanaco/tokenizer.json')

In [17]:
prompt = "Who is Napoleon Bonaparte?"
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200
)
result_fine_tuning = pipe(f'[INST] {prompt} [/INST]')
print(result_fine_tuning[0]['generated_text'])

[INST] Who is Napoleon Bonaparte? [/INST] Napoleon Bonaparte was a French military leader and politician who ruled as Emperor of the French from 1804 to 1814. He was also a general in the French army and a politician who served as President of the French Republic from 1815 to 1824. Napoleon was born in Corsica in 1769 and grew up in France. He began his military career in the French army and rose to the rank of general. Napoleon was a skilled military leader and was known for his courage and determination. He was also a political leader who was able to unite France during the French Revolution and establish the First French Republic. Napoleon was a popular figure in France and was often referred to as "le Grand Duc" or "le Roi de la France". Napoleon was also known for his love of music and theater and was a tal
